In [2]:
from torch.utils.data import Dataset
import torch
import glob
import pandas as pd
from datetime import datetime
import os
import math
from tqdm import tqdm
import numpy as np

from scipy import signal

/home/yhbedoya/VirtualEnvs/SHMmae_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
start_time, end_time = "05/12/2021 00:00", "05/12/2021 00:30"
path = '/home/yhbedoya/Repositories/SHM-MAE/traffic/20211205/'

print(f'reading CSV files')
start = datetime.strptime(start_time, '%d/%m/%Y %H:%M')
end = datetime.strptime(end_time, '%d/%m/%Y %H:%M')

ldf = list()
for p in tqdm(glob.glob(path + "*.csv")):
    name = os.path.split(p)[-1]
    nstr = datetime.strptime(name, 'traffic_%Y%m%dH%H%M%S.csv')
    if start <= nstr < end:
        df_tmp = pd.read_csv(p)
        c_drop = set(df_tmp.columns) - set(["sens_pos", "z", "ts"])
        if len(c_drop) > 0:
            df_tmp.drop(columns=list(c_drop), inplace=True)
        ldf.append(df_tmp)
df = pd.concat(ldf).sort_values(by=['sens_pos', 'ts'])
df.reset_index(inplace=True, drop=True)

#df = df[df['sens_pos'].isin(self.sensors)]
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
data = df

reading CSV files


100%|██████████| 1450/1450 [00:10<00:00, 132.91it/s]


In [4]:
data

,sens_pos,z,ts
0,C1.1.1,-0.988022,2021-12-05 00:00:00.311
1,C1.1.1,-0.988098,2021-12-05 00:00:00.321
2,C1.1.1,-0.988403,2021-12-05 00:00:00.331
3,C1.1.1,-0.987945,2021-12-05 00:00:00.341
4,C1.1.1,-0.988479,2021-12-05 00:00:00.351
...,...,...,...
18894780,C9.2.4,-0.970703,2021-12-05 00:30:00.030
18894781,C9.2.4,-0.971084,2021-12-05 00:30:00.040
18894782,C9.2.4,-0.971313,2021-12-05 00:30:00.050
18894783,C9.2.4,-0.971389,2021-12-05 00:30:00.060


In [5]:
pesaDataDf = pd.read_csv("/home/yhbedoya/Repositories/SHM-MAE/dati_pese_dinamiche/dati 2021-12-04_2021-12-12 pesa km 104,450.csv", sep=";", index_col=0)
pesaDataDf.columns

Index(['Id', 'Ds000IdSite', 'SiteName', 'StartTimeStr', 'LaneNo', 'ClassId',
       'Direction', 'VehicleLength', 'GrossWeight', 'GrossWeightAxle1',
       'GrossWeightAxle2', 'GrossWeightAxle3', 'GrossWeightAxle4',
       'GrossWeightAxle5', 'GrossWeightAxle6', 'GrossWeightAxle7',
       'GrossWeightAxle8', 'GrossWeightAxle9', 'GrossWeightAxle10',
       'GrossWeightAxle11', 'PositionRight', 'PositionLeft', 'Velocity',
       'WheelBase', 'AxlesCount', 'MassUnit', 'VelocityUnit', 'DistanceUnit',
       'TotalLeftImbalance', 'TotalRightImbalance', 'ImbalanceUnit',
       'OverLoad', 'Plate', 'Plate2', 'BetweenLane', 'NamePhoto', 'NamePhoto2',
       'PathPhoto', 'Temperature', 'TemperatureUnit', 'Deflection',
       'DeflectionUnit', 'StrongAccDec'],
      dtype='object')

In [11]:
pesaDataDf = pd.read_csv("/home/yhbedoya/Repositories/SHM-MAE/dati_pese_dinamiche/dati 2021-12-04_2021-12-12 pesa km 104,450.csv", sep=";", index_col=0)
pesaDataDf = pesaDataDf[["Id", "StartTimeStr", "ClassId", "GrossWeight", "Velocity", "VelocityUnit"]]
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["StartTimeStr"])
pesaDataDf["Time"] = pesaDataDf["Time"].dt.strftime('%Y-%d-%m %H:%M:00')
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["Time"]) + pd.to_timedelta(-1,'H')
pesaDataDf["EstimatedTime"] = pesaDataDf["Time"] + pd.to_timedelta(600/(pesaDataDf["Velocity"]/3.6),'S')
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["Time"]).dt.strftime('%Y-%m-%d %H:%M:00')
pesaDataDf[(pesaDataDf["Time"]>="2021-12-05 00:04:00") & (pesaDataDf["Time"]<="2021-12-05 00:12:00")]


,Id,StartTimeStr,ClassId,GrossWeight,Velocity,VelocityUnit,Time,EstimatedTime
N.,,,,,,,,
46817,52381,05/12/2021 01:10,Classe 1,1530,87,Km/h,2021-12-05 00:10:00,2021-12-05 00:10:24.827586207
46818,52380,05/12/2021 01:09,Classe 9,21180,86,Km/h,2021-12-05 00:09:00,2021-12-05 00:09:25.116279070
46819,52379,05/12/2021 01:09,Classe 1,1360,128,Km/h,2021-12-05 00:09:00,2021-12-05 00:09:16.875000000
46820,52378,05/12/2021 01:07,Classe 1,2100,77,Km/h,2021-12-05 00:07:00,2021-12-05 00:07:28.051948052
46821,52377,05/12/2021 01:06,Classe 1,1270,95,Km/h,2021-12-05 00:06:00,2021-12-05 00:06:22.736842105
46822,52376,05/12/2021 01:05,Classe 1,1325,80,Km/h,2021-12-05 00:05:00,2021-12-05 00:05:27.000000000
46823,52375,05/12/2021 01:04,Classe 1,2080,91,Km/h,2021-12-05 00:04:00,2021-12-05 00:04:23.736263736
46824,52374,05/12/2021 01:04,Classe 1,1195,96,Km/h,2021-12-05 00:04:00,2021-12-05 00:04:22.500000000


In [2]:
pesaDataDf = pd.read_csv("/home/yhbedoya/Repositories/SHM-MAE/dati_pese_dinamiche/dati 2021-12-04_2021-12-12 pesa km 104,450.csv", sep=";", index_col=0)
pesaDataDf = pesaDataDf[["Id", "StartTimeStr"]]
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["StartTimeStr"])
pesaDataDf["Time"] = pesaDataDf["Time"].dt.strftime('%Y-%d-%m %H:%M:00')
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["Time"]) + pd.to_timedelta(-1,'H') 
aggDf = pesaDataDf.groupby(["Time"])["Id"].count()
labelsDf = aggDf.reset_index()
labelsDf["Time"] = pd.to_datetime(labelsDf["Time"]).dt.strftime('%Y-%m-%d %H:%M:00')
labelsDf.rename(columns={"Id": "Vehicles"}, inplace=True)
labelsDf

,Time,Vehicles
0,2021-12-03 23:13:00,2
1,2021-12-03 23:32:00,2
2,2021-12-04 02:57:00,1
3,2021-12-04 03:22:00,1
4,2021-12-04 03:50:00,2
...,...,...
9816,2021-12-12 22:48:00,4
9817,2021-12-12 22:52:00,1
9818,2021-12-12 22:54:00,1
9819,2021-12-12 22:55:00,1


In [15]:
labelsDf[(labelsDf["Time"]>="2021-12-04 23:55:00") & (labelsDf["Time"]<="2021-12-05 00:30:00")]

,Time,Vehicles
1081,2021-12-04 23:55:00,3
1082,2021-12-04 23:56:00,1
1083,2021-12-04 23:57:00,1
1084,2021-12-05 00:03:00,3
1085,2021-12-05 00:04:00,2
1086,2021-12-05 00:05:00,1
1087,2021-12-05 00:06:00,1
1088,2021-12-05 00:07:00,1
1089,2021-12-05 00:09:00,2
1090,2021-12-05 00:10:00,1


In [9]:
def _transformation(slice):
    sliceN = slice-torch.mean(slice)
    frequencies, times, spectrogram = signal.spectrogram(sliceN,sampleRate,nfft=frameLength,noverlap=(frameLength - stepLength), nperseg=frameLength,mode='psd')

    return frequencies, times, np.log10(spectrogram)

def _normalizer(spectrogram):
    spectrogramNorm = (spectrogram - min) / (max - min)
    return spectrogramNorm

def butter_bandpass(lowcut, highcut, fs, order=5):
    return signal.butter(order, [1, 49], fs=fs, btype='band')

def butter_bandpass_filter(slice, lowcut, highcut, fs, order=5):
    sliceN = slice-np.mean(np.array(slice))
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, sliceN)
    return y

In [43]:
sampleRate = 100
frameLength = 256
stepLength = 64
windowLength= 6000
windowStep = 1500

sensors = data['sens_pos'].unique().tolist()
print(f'start partitioner')
partitions = {}
cumulatedWindows = 0
limits = dict()
print(f'Generating windows')
for sensor in tqdm(sensors):
    if sensor != "C1.1.2":
        continue
    sensorData = data[data['sens_pos']==sensor]
    totalFrames = sensorData.shape[0]
    totalWindows = math.ceil((totalFrames-windowLength)/windowStep)
    start = cumulatedWindows
    cumulatedWindows += totalWindows
    end = cumulatedWindows
    indexStart = sensorData.index[0]
    partitions[sensor]= (start, end, indexStart)

timeData = torch.tensor(data["z"].values, dtype=torch.float64)
timestamps =  np.array(data["ts"].values)
#vehiclesData = data["Vehicles"]
cummulator = -1
posCummulator = 0
negCummulator = 0


mins = list()
maxs = list()
print(f'Defining useful windows limits')
noiseFreeSpaces = 1
for index in tqdm(range(0, cumulatedWindows)):
    for k,v in partitions.items():
        if index in range(v[0], v[1]):
            start = v[2]+(index-v[0])*windowStep
            filteredSlice = butter_bandpass_filter(timeData[start: start+windowLength], 0, 50, sampleRate)
            amp = np.max(filteredSlice)-np.min(filteredSlice)
            if True:
                posCummulator +=1 
                cummulator += 1
                #label = vehiclesData[start : start+windowLength].mean()
                limits[cummulator] = (start, start+windowLength, amp, timestamps[start])
                slice = timeData[start:start+windowLength]
                frequencies, times, spectrogram = _transformation(torch.tensor(slice, dtype=torch.float64))
                mins.append(np.min(np.array(spectrogram)))
                maxs.append(np.max(np.array(spectrogram)))
                noiseFreeSpaces += 1
                
            elif noiseFreeSpaces>0:
                negCummulator +=1
                cummulator += 1
                #label = vehiclesData[start : start+windowLength].mean()
                limits[cummulator] = (start, start+windowLength, amp)
                slice = timeData[start:start+windowLength]
                frequencies, times, spectrogram = _transformation(torch.tensor(slice, dtype=torch.float64))
                mins.append(np.min(np.array(spectrogram)))
                maxs.append(np.max(np.array(spectrogram)))
                noiseFreeSpaces -= 1
            break
print(f'Total windows in dataset: {cummulator}')
min = np.min(np.array(mins))
max = np.max(np.array(maxs))
print(f'Total positive instances: {posCummulator}')
print(f'Total noisy instances: {negCummulator}')
print(f'Proportion of useful instances {(posCummulator+negCummulator)/cumulatedWindows}')       
print(f'General min: {min}')
print(f'General max: {max}')

start partitioner
Generating windows


100%|██████████| 105/105 [00:00<00:00, 176.88it/s]


Defining useful windows limits


100%|██████████| 116/116 [00:00<00:00, 928.60it/s]

Total windows in dataset: 115
Total positive instances: 116
Total noisy instances: 0
Proportion of useful instances 1.0
General min: -17.353365175742226
General max: -4.353973361509182


In [24]:
def __getitem__(index):
    start, end, std, timestamp = limits[index]
    slice = timeData[start:end]
    frequencies, times, spectrogram = _transformation(slice)
    spectrogram = torch.unsqueeze(torch.tensor(spectrogram, dtype=torch.float64), 0)
    
    return frequencies, times, spectrogram, std, timestamp


In [38]:
import matplotlib.pyplot as plt

def plotSpect(frequencies, times, spectrogram, index, std, timestamp):
    plt.figure(figsize=(10, 5))
    plt.title(f'spectrogram from PSD: {round(std, 4)} Time: {timestamp}')
    plt.pcolormesh(times, frequencies, 10*(np.squeeze(spectrogram)), vmin=-150, vmax=-50)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(format="%+2.f", label='dB')
    #folder = "positives" if std > 0.0075 else "noise"
    plt.savefig(f'/home/yhbedoya/Repositories/SHM-MAE/LabelTest/C1.1.2/{index}.png')
    plt.close()

In [44]:
for i in range(len(limits)):
    frequencies, times, spectrogram, std, timestamp = __getitem__(i)
    plotSpect(frequencies, times, spectrogram, i, std, timestamp)